In [1]:
# !pip install autocorrect

In [2]:
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import PorterStemmer
from autocorrect import Speller

import string
import unicodedata
import os
from six.moves import cPickle
import re

In [3]:
import tensorflow as tf
from tensorboard.plugins import projector
sconfig = projector.ProjectorConfig()

### LOAD DATA

In [4]:
data = pd.read_csv('frames.csv')

In [5]:
data.head()

,0,1
0,I'd like to book a trip to Atlantis from Capri...,"Hi...I checked a few options for you, and unfo..."
1,"Yes, how about going to Neverland from Caprica...",I checked the availability for this date and t...
2,I have no flexibility for dates... but I can l...,I checked the availability for that date and t...
3,I suppose I'll speak with my husband to see if...,NaN
4,"Hello, I am looking to book a vacation from Go...","Hi. Sorry, I can't find any trips from Gotham ..."


In [6]:
data.isnull().sum()

0      0
1    828
dtype: int64

In [7]:
len(data)

10407

In [8]:
data = data.dropna()

In [9]:
len(data)

9579

In [10]:
data.head()

,0,1
0,I'd like to book a trip to Atlantis from Capri...,"Hi...I checked a few options for you, and unfo..."
1,"Yes, how about going to Neverland from Caprica...",I checked the availability for this date and t...
2,I have no flexibility for dates... but I can l...,I checked the availability for that date and t...
4,"Hello, I am looking to book a vacation from Go...","Hi. Sorry, I can't find any trips from Gotham ..."
5,What about a trip from Gotham City to Neverlan...,"Sorry, I cannot find any trips leaving from Go..."


In [11]:
data.rename(columns = {'0':'Questions','1':'Answers'}, inplace = True)

<ipython-input-11-6e455cb74bc9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {'0':'Questions','1':'Answers'}, inplace = True)


In [12]:
data.head()

,Questions,Answers
0,I'd like to book a trip to Atlantis from Capri...,"Hi...I checked a few options for you, and unfo..."
1,"Yes, how about going to Neverland from Caprica...",I checked the availability for this date and t...
2,I have no flexibility for dates... but I can l...,I checked the availability for that date and t...
4,"Hello, I am looking to book a vacation from Go...","Hi. Sorry, I can't find any trips from Gotham ..."
5,What about a trip from Gotham City to Neverlan...,"Sorry, I cannot find any trips leaving from Go..."


In [13]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [14]:
spell = Speller(lang='en')

def process_str(string):
    string = unicode_to_ascii(string.lower().strip())
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`:]", " ", string)
    string = re.sub(r"\'s", " \'s",string)
    string = re.sub(r"\'ve", " \'ve",string)
    string = re.sub(r"n\'t", " n\'t",string)
    string = re.sub(r"\'re", " \'re",string)
    string = re.sub(r"\'d", " \'d",string)
    string = re.sub(r"\'ll", " \'ll",string)
    string = re.sub(r",", " , ",string)
    string = re.sub(r"!", " ! ",string)
    string = re.sub(r"\s{2,}", " ",string)
    string = re.sub("(\\W)"," ",string) 
    string = re.sub('\S*\d\S*\s*','', string)
    string =  "<sos> " +  string + " <eos>"
    return string

In [15]:
data["Questions"][:5]

0    I'd like to book a trip to Atlantis from Capri...
1    Yes, how about going to Neverland from Caprica...
2    I have no flexibility for dates... but I can l...
4    Hello, I am looking to book a vacation from Go...
5    What about a trip from Gotham City to Neverlan...
Name: Questions, dtype: object

In [16]:
data["Questions"] = data.Questions.apply(process_str)

In [17]:
data["Questions"][:5]

0    <sos> i  d like to book a trip to atlantis fro...
1    <sos> yes   how about going to neverland from ...
2    <sos> i have no flexibility for dates but i ca...
4    <sos> hello   i am looking to book a vacation ...
5    <sos> what about a trip from gotham city to ne...
Name: Questions, dtype: object

In [18]:
Questions = data.Questions.values.tolist()
Answers = data.Answers.values.tolist()